# DQN

본 노트북에서는 잘 알려진 알고리즘인 DQN알고리즘을 활용하여,

competitive setting에서의 학습을 진행해보겠습니다.

ML2에서 자체 개발한 `rl2`라이브러리를 사용하여 학습 해보겠습니다.

rl2의 자세한 사용법은 https://github.com/kc-ml2/rl2 를 참고 부탁드립니다.

## 셋업

In [1]:
## 패키지 설치

In [2]:
# !pip3 intsall marlenv rl2

In [3]:
## 패키지 로드

In [4]:
%load_ext tensorboard

In [5]:
import sys
from pprint import pprint

### 학습 디바이스 세팅

In [6]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## Env

In [7]:
from marlenv.wrappers import make_snake

In [8]:
env_config = dict(
    num_snakes=1,
    n_env=1,
    height=20,
    width=20,
    frame_stack=2,
    vision_range=5,
)

In [9]:
env, obs_shape, ac_shape, props = make_snake(**env_config)

## Config

In [10]:
config = dict(
    width=20, 
    height=20, 
    vision_range=5, 
    frame_stack=2,
    buffer_size=int(1e5), 
    batch_size=32, 
    num_epochs=1,
    max_step=int(5e5), 
    update_interval=int(1e4), 
    train_interval=1,
    log_interval=20000, 
    save_interval=int(1e5),
    optimizer='torch.optim.Adam', 
    lr=1e-3, 
    recurrent=True,
    gamma=0.99, 
    eps=0.001, 
    polyak=0, 
    decay_step=int(1e5),
    grad_clip=10, 
    tag='DDQN', 
    double=True,
    log_level=10
)

In [11]:
# TODO: remove dependency on easydict
from easydict import EasyDict
config=EasyDict(config)

## Model

In [12]:
from rl2.agents.dqn import DQNModel

In [13]:
model = DQNModel(
    observation_shape=obs_shape,
    action_shape=ac_shape,
    double=config.double,
    recurrent=config.recurrent,
    optimizer=config.optimizer,
    lr=config.lr,
    grad_clip=config.grad_clip,
    polyak=config.polyak,
    reorder=True,
    discrete=props.discrete,
    high=props.high
)

## Agent

`rl2`의 predefined model인 `PPOAgent` 클래스를 사용하여 학습하겠습니다.

In [14]:
from rl2.agents.dqn import DQNAgent

In [15]:
agent = DQNAgent(
    model,
    update_interval=config.update_interval,
    train_interval=config.train_interval,
    num_epochs=config.num_epochs,
    buffer_size=config.buffer_size,
    batch_size=config.batch_size,
    decay_step=config.decay_step,
    eps=config.eps,
    gamma=config.gamma,
    log_interval=config.log_interval,
)

## Train

In [16]:
from rl2.workers.multi_agent import MaxStepWorker

### Logger

`rl2`에서는 본인이 원하는 custom logger 클래스를 정의하여 로깅을 할 수 있습니다.

예시로 제공하는 `Logger` 클래스를 활용하여 로깅을 해보겠습니다.

해당 클래스는 tensorboard의 `FileWriter`객체를 가지고 있습니다.

In [17]:
from rl2.examples.temp_logger import Logger

로깅 componenets
* checkpoint
* tensorboard summary data
* env setting
* etc

In [18]:
config['log_dir'] = './DQN'
logger = Logger(name='DQN', args=config)

/home/anthony/.pyenv/versions/3.7.9/envs/dev/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [21]:
worker = MaxStepWorker(
    env, 
    props.n_env, 
    agent,
    max_steps=config.max_step, 
    training=True,
    log_interval=config.log_interval,
    render=True,
    render_mode='rgb_array',
    render_interval=10000,
    is_save=True,
    save_interval=config.save_interval,
    logger=logger
)

In [ ]:
worker.run()

  ----------------------------------
  | Counts/num_episodes | 900      |
  | Counts/num_steps    | 20000    |
  | Episodic/ep_length  | 22.4     |
  | Episodic/rews_avg   | 0.17     |
  ----------------------------------
  ----------------------------------
  | Counts/num_episodes | 1796     |
  | Counts/num_steps    | 40000    |
  | Episodic/ep_length  | 25.7     |
  | Episodic/rews_avg   | 0.35     |
  ----------------------------------
  ----------------------------------
  | Counts/num_episodes | 2613     |
  | Counts/num_steps    | 60000    |
  | Episodic/ep_length  | 22.8     |
  | Episodic/rews_avg   | 0.62     |
  ----------------------------------
  ----------------------------------
  | Counts/num_episodes | 3405     |
  | Counts/num_steps    | 80000    |
  | Episodic/ep_length  | 25.9     |
  | Episodic/rews_avg   | 0.73     |
  ----------------------------------
  ----------------------------------
  | Counts/num_episodes | 3735     |
  | Counts/num_steps    | 100000   |
 

## 텐서보드

In [ ]:
# %tensorboard --logdir .

## 에이전트 검증

In [ ]:
!python3 -m pytest support/test.py

## 에이전트 제출

In [ ]:
!sh submit.sh